In [1]:
#Celda para librerías
import sklearn as sk
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from tqdm import tqdm_notebook as tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler
import json
import warnings
warnings.filterwarnings('ignore')

print(np.__version__)
from ydata_profiling import ProfileReport
import os
print("Directorio actual:", os.getcwd())

# Have all columns appear when dataframes are displayed.
pd.set_option('display.max_columns', None) 
# Have 100 rows appear when a dataframe is displayed
pd.set_option('display.max_rows', 500)
# Display dimensions whenever a dataframe is printed out.
pd.set_option('display.show_dimensions', True)

#Importando los datos

app_train = pd.read_csv(r'C:/Users/Yeray/Desktop/DATA_SCIENCE_ML/Home-Credit-TFG/DATA/home-credit-default-risk/application_train.csv')
bureau_final = pd.read_csv(r'C:/Users/Yeray/Desktop/DATA_SCIENCE_ML/Home-Credit-TFG/DATA/bureau_agg_final.csv')
bureau_balance__agg_def_v1 = pd.read_csv(r'C:/Users/Yeray/Desktop/DATA_SCIENCE_ML/Home-Credit-TFG/DATA/bureau_balance_agg_def_v2.csv')
print(app_train.shape
      , bureau_final.shape
      , bureau_balance__agg_def_v1.shape)


1.23.5
Directorio actual: c:\Users\Yeray\Desktop\DATA_SCIENCE_ML\Home-Credit-TFG\JUPYTER_NOTEBOOKS


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Yeray/Desktop/DATA_SCIENCE_ML/Home-Credit-TFG/DATA/bureau_balance_agg_def_v2.csv'

In [ ]:
#Uniendo los datos
app_train = app_train.merge(bureau_final, on='SK_ID_CURR', how='left')
app_train = app_train.merge(bureau_balance__agg_def_v1, on='SK_ID_CURR', how='left')

app_train.drop(columns=['SK_ID_CURR'], inplace=True)
#Debido a un fallo que detecta que todas los valores de la columna DAYS_BIRTH 
# son nan cuando no es así, vamos a sustituir la columna por una nueva llamada AGE_INT
app_train['AGE_INT'] = app_train['DAYS_BIRTH']/-365
app_train['HAS_CHILDREN'] = np.where(app_train['CNT_CHILDREN'] > 0, 1, 0)
#variables_categoricas = app_train.select_dtypes(include=['object', 'category']).columns
#app_train[variables_categoricas] = app_train[variables_categoricas].astype('category')
#categoricas = variables_categoricas.tolist()
info = app_train.info()
print(info)
#describe = app_train.describe()
#print(describe)

In [4]:
#Feature Engineering


#Vamos a convertir la variable CNT_CHILDREN en una variable categórica que indique 1 si tiene hijos y 0 si no.

#Vamos a crear la variable DAYS_EMPLOYED en una variable categórica que indique 1 si tiene empleo y 0 si no.
app_train['HAS_EMPLOYMENT'] = app_train['DAYS_EMPLOYED'].map(lambda app_train: 1 if app_train < 0 else 0)
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)


#Voy a crear RATIOS para algunas de las variables más importantes (en mi opinión y según lo investigado) 
# y que además tienen valores atípicos.
app_train['CREDIT_AMT_INCOME_RATIO'] = app_train['AMT_CREDIT']/app_train['AMT_INCOME_TOTAL']
app_train['CREDIT_AMT_INCOME_PORCENTAJE'] = app_train['AMT_INCOME_TOTAL']/app_train['AMT_CREDIT']

app_train['ANNUITY_AMT_INCOME_RATIO'] = app_train['AMT_INCOME_TOTAL']/app_train['AMT_ANNUITY']
app_train['ANNUITY_INCOME_PORCENTAJE'] = app_train['AMT_ANNUITY']/app_train['AMT_INCOME_TOTAL']

app_train['GOODS_INCOME_RATIO'] = app_train['AMT_GOODS_PRICE']/app_train['AMT_INCOME_TOTAL']

app_train['CREDIT_ANNUITY_RATIO'] = app_train['AMT_CREDIT']/app_train['AMT_ANNUITY']
app_train['ANNUITY_CREDIT_PORCETAJE'] = app_train['AMT_ANNUITY']/app_train['AMT_CREDIT']
app_train['CREDIT_GOODS_PRICE_RATIO'] = app_train['AMT_CREDIT']/app_train['AMT_GOODS_PRICE']

app_train['ANNUITY_TO_DAYS_EMPLOYED_RATIO'] = app_train['AMT_ANNUITY']/app_train['DAYS_EMPLOYED']
app_train['ANNUITY_TO_CHILDREN_RATIO'] = app_train['AMT_ANNUITY']/app_train['CNT_CHILDREN']

app_train['INCOME_PER_CHILD'] = app_train['AMT_INCOME_TOTAL']/app_train['CNT_CHILDREN']
app_train['CREDIT_PER_CHILD'] = app_train['AMT_CREDIT']/app_train['CNT_CHILDREN']

app_train['AMT_INCOME_FAMILY_MEMBERS_RATIO'] = app_train['AMT_INCOME_TOTAL']/app_train['CNT_FAM_MEMBERS']
app_train['CHILDREN_RATIO'] = app_train['CNT_CHILDREN']/app_train['CNT_FAM_MEMBERS']
app_train['ADULTS_MEMBERS_DIFF'] = app_train['CNT_FAM_MEMBERS'] - app_train['CNT_CHILDREN']

app_train['INCOME_PER_ADULT'] = app_train['AMT_INCOME_TOTAL']/app_train['ADULTS_MEMBERS_DIFF']
app_train['CREDIT_PER_MEMBER'] = app_train['AMT_CREDIT']/app_train['CNT_FAM_MEMBERS']
app_train['CREDIT_PER_ADULT'] = app_train['AMT_CREDIT']/app_train['ADULTS_MEMBERS_DIFF']
app_train['ANNUITY_PER_ADULT'] = app_train['AMT_ANNUITY']/app_train['ADULTS_MEMBERS_DIFF']


#app_train['OWN_CAR_AGE_RATIO'] = app_train['OWN_CAR_AGE']/app_train['AGE_INT']
app_train['NEW_CAR_BIRTH_RATIO'] = app_train['OWN_CAR_AGE']/app_train['DAYS_BIRTH']
app_train['NEW_CAR_EMPLOYMENT_RATIO'] = app_train['OWN_CAR_AGE']/app_train['DAYS_EMPLOYED']


app_train['EMPLOYMENT_AGE_RATIO'] = app_train['DAYS_EMPLOYED']/-app_train['AGE_INT']
#app_train['EMPLOYMENT_BIRTH_RATIO'] = app_train['DAYS_EMPLOYED']/app_train['DAYS_BIRTH']
app_train['EMPLOYMENT_PHONE_CHANGE_RATIO'] = app_train['DAYS_LAST_PHONE_CHANGE']/app_train['DAYS_EMPLOYED']

app_train['NEW_PHONE_CHANGE_BIRTH_RATIO'] = app_train['DAYS_LAST_PHONE_CHANGE']/app_train['DAYS_BIRTH']
app_train['NEW_PHONE_CHANGE_EMPLOYMENT_RATIO'] = app_train['DAYS_LAST_PHONE_CHANGE']/app_train['DAYS_EMPLOYED']
app_train['DAYS_ID_PUBLISH_BIRTH_RAIO'] = app_train['DAYS_ID_PUBLISH']/app_train['DAYS_BIRTH']

app_train['REGISTRATION_TO_NEW_PHONE_CHANGE_RATIO'] = app_train['DAYS_LAST_PHONE_CHANGE']/app_train['DAYS_REGISTRATION']
#app_train['REGISTRATION_TO_ID_PUBLISH_RATIO'] = app_train['DAYS_ID_PUBLISH'] - app_train['DAYS_REGISTRATION']
#app_train['REGISTRATION_TO_BIRTH_RATIO'] = app_train['DAYS_BIRTH'] - app_train['DAYS_REGISTRATION']

#EXT_SOURCE son variables representan fuentes externas de información sobre el cliente, y suelen estar relacionadas con puntuaciones de riesgo crediticio generadas por 
# instituciones externas al prestamista, como burós de crédito u otras entidades que evalúan el perfil financiero de los clientes.
#Vamos a crear variables relacionadas con EXT_SOURCE.

app_train['EXT_SOURCE_SUM'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].sum(axis=1)
app_train['EXT_SOURCE_MEAN'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
app_train['EXT_SOURCE_MEDIAN'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].median(axis=1)
app_train['EXT_SOURCE_MAX'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].max(axis=1)
app_train['EXT_SOURCE_MIN'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].min(axis=1)

app_train['EXT_SOURCE_WEIGHTED_SUM'] = app_train['EXT_SOURCE_1']*3 + app_train['EXT_SOURCE_2']*1 + app_train['EXT_SOURCE_3']*5
app_train['EXT_SOURCE_WEIGHTED_MEAN'] = (app_train['EXT_SOURCE_1']*3 + app_train['EXT_SOURCE_2']*1 + app_train['EXT_SOURCE_3']*5)/3
app_train['EXT_SOURCE_PROD'] = app_train['EXT_SOURCE_1']*app_train['EXT_SOURCE_2']*app_train['EXT_SOURCE_3']

app_train['RATIO_EXT_SOURCE_3_TO_REGION_POPULATION_RELATIVE'] = app_train['EXT_SOURCE_3'] / app_train['REGION_POPULATION_RELATIVE']

"""
app_train['AGE_CREDIT_RATIO'] = app_train['AGE_INT']/app_train['AMT_CREDIT']
app_train['AGE_ANNUITY_RATIO'] = app_train['AGE_INT']/app_train['AMT_ANNUITY']
app_train['AGE_GOODS_PRICE_RATIO'] = app_train['AGE_INT']/app_train['AMT_GOODS_PRICE']
app_train['AGE_INCOME_RATIO'] = app_train['AGE_INT']/app_train['AMT_INCOME_TOTAL']
"""
app_train['AGE_CREDIT_RATIO'] = app_train['AMT_CREDIT']/app_train['DAYS_BIRTH']
app_train['AGE_ANNUITY_RATIO'] = app_train['AMT_ANNUITY']/app_train['DAYS_BIRTH']
app_train['AGE_GOODS_PRICE_RATIO'] = app_train['AMT_GOODS_PRICE']/app_train['DAYS_BIRTH']
app_train['AGE_INCOME_RATIO'] = app_train['AMT_INCOME_TOTAL']/app_train['DAYS_BIRTH']



#Variables a partir de sumas y diferencias
app_train['AMT_INCOME_TOTAL_ANNUITY_SUM'] = app_train['AMT_INCOME_TOTAL'] + app_train['AMT_ANNUITY']
app_train['AMT_GOODS_TO_INCOME_ANUITY_SUM_RATIO'] = app_train['AMT_GOODS_PRICE']/(app_train['AMT_INCOME_TOTAL_ANNUITY_SUM'] )
app_train['CREDIT_BUREAU_TOTAL'] = app_train[['AMT_REQ_CREDIT_BUREAU_DAY', 
                                             'AMT_REQ_CREDIT_BUREAU_HOUR',
                                             'AMT_REQ_CREDIT_BUREAU_MON',
                                             'AMT_REQ_CREDIT_BUREAU_QRT',
                                             'AMT_REQ_CREDIT_BUREAU_WEEK',
                                             'AMT_REQ_CREDIT_BUREAU_YEAR']].sum(axis=1)

app_train['CREDIT_GOODS_DIFF'] = app_train['AMT_GOODS_PRICE'] - app_train['AMT_CREDIT']
app_train['GOODS_ANNUITY_DIFF'] = app_train['AMT_ANNUITY'] - app_train['AMT_GOODS_PRICE']
app_train['AMT_INCOME_TOTAL_ANNUITY_DIFF'] = app_train['AMT_INCOME_TOTAL'] - app_train['AMT_ANNUITY']

#app_train['SOCIAL_OBSERVATION_TOTAL'] = app_train['OBS_30_CNT_SOCIAL_CIRCLE'] + app_train['OBS_60_CNT_SOCIAL_CIRCLE']
#app_train['SOCIAL_DEF_TOTAL'] = app_train['DEF_30_CNT_SOCIAL_CIRCLE'] + app_train['DEF_60_CNT_SOCIAL_CIRCLE']

app_train['DIFF_OBS_30_60'] = app_train['OBS_30_CNT_SOCIAL_CIRCLE'] - app_train['OBS_60_CNT_SOCIAL_CIRCLE']
app_train['DIFF_DEF_30_60'] = app_train['DEF_30_CNT_SOCIAL_CIRCLE'] - app_train['DEF_60_CNT_SOCIAL_CIRCLE'] 

app_train['LONG_EMPLOYMENT'] = np.where(app_train['DAYS_EMPLOYED'] < -2000, 1, 0)
app_train['RETIRED'] = np.where(app_train['DAYS_BIRTH'] < -14000, 1, 0)

#OTRAS






In [5]:
education_mapping = {
    'No formal education': 0,
    'Lower secondary': 1,
    'Secondary / secondary special': 2,
    'Incomplete higher': 3,
    'Higher education': 4,
    'Academic degree': 5
}

app_train['EDUCATION_LEVEL'] = app_train['NAME_EDUCATION_TYPE'].map(education_mapping)
print(app_train['EDUCATION_LEVEL'].describe())
"""
income_type_mapping = {
    'Unemployed': 0,
    'Student': 1,
    'Pensioner': 2,
    'Maternity leave': 3,
    'Working': 4,
    'State servant': 5,
    'Commercial associate': 6,
    'Businessman': 7,    
}
app_train['INCOME_TYPE'] = app_train['NAME_INCOME_TYPE'].map(income_type_mapping)

family_mapping = {
    'Single / not married': 0,
    'Separated': 1,
    'Widow': 2,
    'Married': 3
}

housing_mapping = {
    'With parents': 0,
    'Municipal apartment': 1,
    'Rented apartment': 2,
    'House / apartment': 3
}



app_train['FAMILY_STATUS'] = app_train['NAME_FAMILY_STATUS'].map(family_mapping)
app_train['HOUSING_TYPE'] = app_train['NAME_HOUSING_TYPE'].map(housing_mapping)

"""

count    307511.000000
mean          2.509507
std           0.874544
min           1.000000
25%           2.000000
50%           2.000000
75%           3.000000
max           5.000000
Name: EDUCATION_LEVEL, Length: 8, dtype: float64


"\nincome_type_mapping = {\n    'Unemployed': 0,\n    'Student': 1,\n    'Pensioner': 2,\n    'Maternity leave': 3,\n    'Working': 4,\n    'State servant': 5,\n    'Commercial associate': 6,\n    'Businessman': 7,    \n}\napp_train['INCOME_TYPE'] = app_train['NAME_INCOME_TYPE'].map(income_type_mapping)\n\nfamily_mapping = {\n    'Single / not married': 0,\n    'Separated': 1,\n    'Widow': 2,\n    'Married': 3\n}\n\nhousing_mapping = {\n    'With parents': 0,\n    'Municipal apartment': 1,\n    'Rented apartment': 2,\n    'House / apartment': 3\n}\n\n\n\napp_train['FAMILY_STATUS'] = app_train['NAME_FAMILY_STATUS'].map(family_mapping)\napp_train['HOUSING_TYPE'] = app_train['NAME_HOUSING_TYPE'].map(housing_mapping)\n\n"

In [6]:
#AGREGACIONES
agregaciones = [
    (['CODE_GENDER', 'NAME_EDUCATION_TYPE'], [('AMT_ANNUITY', 'max'),
                                              ('AMT_CREDIT', 'max'),
                                              ('EXT_SOURCE_1', 'mean'),
                                              ('EXT_SOURCE_2', 'mean'),
                                              ('EXT_SOURCE_3', 'mean'),
                                              ('AGE_INT', 'mean'),
                                              ('OWN_CAR_AGE', 'max')]),

    (['CODE_GENDER', 'ORGANIZATION_TYPE'], [('AMT_ANNUITY', 'mean'),
                                            ('AMT_CREDIT', 'mean'),
                                            ('EXT_SOURCE_1', 'mean'),
                                            ('EXT_SOURCE_2', 'mean'),
                                            ('EXT_SOURCE_3', 'mean')]),
    (['CODE_GENDER', 'REG_CITY_NOT_WORK_CITY'], [('AMT_ANNUITY', 'mean'),
                                                ('DAYS_LAST_PHONE_CHANGE', 'mean'),
                                                ('DAYS_REGISTRATION', 'mean'),
                                                ('CNT_CHILDREN', 'mean')]),

                                              
                                                                                                                                             
]
groupby_aggregate_names = []
for groupby_cols, specs in tqdm(agregaciones):
    group_object = app_train.groupby(groupby_cols)
    for select, agg in tqdm(specs):
        groupby_aggregate_name = '{}_{}_{}'.format('_'.join(groupby_cols), agg, select)
        app_train = app_train.merge(group_object[select]
                              .agg(agg)
                              .reset_index()
                              .rename(index=str,
                                      columns={select: groupby_aggregate_name})
                              [groupby_cols + [groupby_aggregate_name]],
                              on=groupby_cols,
                              how='left')
        groupby_aggregate_names.append(groupby_aggregate_name)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
# Preprocesamiento previo a la creación de los modelos
# Imputación de valores nulos. Vamos a probar de momento a completar variables numéricos con la media
# y las categóricas con la moda.
# Como sabemos que DAYS_EMPLOYEED tiene un valor erroneo, vamos a reemplazarlo por NaN
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)
app_train['DAYS_LAST_PHONE_CHANGE'].replace({0: np.nan}, inplace = True)
app_train['NAME_FAMILY_STATUS'].replace('Unknown', np.nan, inplace=True)
# Reemplazar valores infinitos por NaN
app_train.replace([np.inf, -np.inf], np.nan, inplace=True)
# Algunas variables categóricas tienen valores como XNA, vamos a reemplazarlos por NaN
def reemplazar_xna_por_nan(app_train):
    variables_categoricas = app_train.select_dtypes(include=['object', 'category']).columns
    for col in variables_categoricas:
        app_train[col] = app_train[col].replace('XNA', np.nan)
    return app_train
app_train_noxna = reemplazar_xna_por_nan(app_train)

app_train = reemplazar_xna_por_nan(app_train)
def imputar_valores_nulos(app_train):
    variables_continuas = app_train.select_dtypes(include=['int64', 'float64']).columns 
    for col in variables_continuas:
        app_train[col] = app_train[col].fillna(app_train[col].mean())
    variables_categoricas = app_train.select_dtypes(include=['object', 'category']).columns
    for col in variables_categoricas:
        app_train[col] = app_train[col].fillna(app_train[col].mode()[0])

    return app_train

app_train_nonan = imputar_valores_nulos(app_train_noxna)
#Comprobamos que no haya valores nulos
#print(app_train['DAYS_BIRTH'].unique())
#print(app_train['DAYS_BIRTH'].isnull().sum())
#print(app_train['DAYS_BIRTH'].dtype)

# Verificar si hay valores nulos en app_train después de la imputación
print("Valores nulos en app_train después de la imputación:")
print(app_train_nonan.isnull().sum().sum())

Valores nulos en app_train después de la imputación:
0


In [8]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Selección de variables categóricas
variables_categoricas = app_train.select_dtypes(include=['object', 'category']).columns
variables_categoricas_binarias = [col for col in variables_categoricas if app_train[col].nunique() == 2]
variables_categoricas_no_binarias = [col for col in variables_categoricas if app_train[col].nunique() > 2]

# Diccionario para almacenar los LabelEncoders
label_encoders_binarias = {}

# Función para ajustar el LabelEncoder a las variables binarias
def label_encoding_binarias_fit(app_train):
    for col in variables_categoricas_binarias:
        Encoder = LabelEncoder()
        Encoder.fit(app_train[col])
        label_encoders_binarias[col] = Encoder

# Función para transformar las variables binarias utilizando los LabelEncoders ajustados
def label_encoding_binarias_transform(data):
    data = data.copy()  # Evitar modificar el original
    for col in variables_categoricas_binarias:
        if col in label_encoders_binarias:
            data[col] = label_encoders_binarias[col].transform(data[col])
    return data

# Función para aplicar One-Hot Encoding a las variables no binarias
def one_hot_encoding_no_binarias(data):
    data = data.copy()  # Evitar modificar el original
    data = pd.get_dummies(data, columns=variables_categoricas_no_binarias)
    return data

# Ajustar los LabelEncoders al conjunto de entrenamiento
label_encoding_binarias_fit(app_train)

# Transformar el conjunto de entrenamiento
app_train_nonan_le = label_encoding_binarias_transform(app_train)

# Aplicar One-Hot Encoding al conjunto de entrenamiento
app_train_nonan_le_oh = one_hot_encoding_no_binarias(app_train_nonan_le)

print("Valores nulos en app_train después de la imputación:")
print(app_train_nonan_le_oh.isnull().sum().sum())
print(app_train_nonan_le_oh.info())


Valores nulos en app_train después de la imputación:
0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 720 entries, TARGET to LATEST_CREDIT_ACTIVE_CAT_(BUREAU)_Sold
dtypes: bool(148), float64(523), int32(8), int64(41)
memory usage: 1.3 GB
None


In [9]:
from sklearn.preprocessing import RobustScaler, MinMaxScaler
import pandas as pd

# Identificar las variables continuas que necesitan normalización
variables_continuas = app_train_nonan_le_oh.select_dtypes(include=['int64', 'float64']).columns
variables_normalizadas = [col for col in variables_continuas if
                          app_train_nonan_le_oh[col].min() >= 0 and app_train_nonan_le_oh[col].max() <= 1]
variables_NO_normalizadas = [col for col in variables_continuas if col not in variables_normalizadas]

# Diccionario para almacenar los escaladores ajustados
scalers = {}

# Función para ajustar el escalador a las variables no normalizadas
def normalizacion_fit(data, cols, scaler_type='minmax'):
    for col in cols:
        if scaler_type == 'minmax':
            scaler = MinMaxScaler()
        elif scaler_type == 'robust':
            scaler = RobustScaler()
        else:
            raise ValueError("Tipo de escalador no reconocido. Usa 'minmax' o 'robust'.")
        scaler.fit(data[[col]])
        scalers[col] = scaler

# Función para transformar las variables utilizando los escaladores ajustados
def normalizacion_transform(data, cols):
    data = data.copy()  # Evitar modificar el original
    for col in cols:
        if col in scalers:
            data[col] = scalers[col].transform(data[[col]])
    return data

# Ajustar los escaladores a las variables no normalizadas en el conjunto de entrenamiento
normalizacion_fit(app_train_nonan_le_oh, variables_NO_normalizadas, scaler_type='minmax')

# Transformar el conjunto de entrenamiento
app_train_nonan_le_oh_norm = normalizacion_transform(app_train_nonan_le_oh, variables_NO_normalizadas)

# Verificar los valores nulos y la información del DataFrame transformado
print("Valores nulos en app_train después de la imputación:")
print(app_train_nonan_le_oh_norm.isnull().sum().sum())
print(app_train_nonan_le_oh_norm.info())
print(app_train_nonan_le_oh_norm.max())

Valores nulos en app_train después de la imputación:
0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 720 entries, TARGET to LATEST_CREDIT_ACTIVE_CAT_(BUREAU)_Sold
dtypes: bool(148), float64(530), int32(8), int64(34)
memory usage: 1.3 GB
None
TARGET                                                     1
NAME_CONTRACT_TYPE                                         1
CODE_GENDER                                                1
FLAG_OWN_CAR                                               1
FLAG_OWN_REALTY                                            1
                                                        ... 
LATEST_CREDIT_TYPE_CAT_(BUREAU)_Unknown type of loan    True
LATEST_CREDIT_ACTIVE_CAT_(BUREAU)_Active                True
LATEST_CREDIT_ACTIVE_CAT_(BUREAU)_Bad debt              True
LATEST_CREDIT_ACTIVE_CAT_(BUREAU)_Closed                True
LATEST_CREDIT_ACTIVE_CAT_(BUREAU)_Sold                  True
Length: 720, dtype: object


In [10]:
#Exportamos el csv tras todo el preprocesamiento
app_train_nonan_le_oh_norm.to_csv(r'C:/Users/Yeray/Desktop/DATA_SCIENCE_ML/Home-Credit-TFG/DATA/application_train_preprocesado_definitivo_v3.csv', index=False)